## Import modules

In [1]:
import numpy as np
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
import os
import time

## Utils

In [2]:
def load_matrix(path: str, verbose: bool = True) -> np.ndarray:
    matrix = np.load(path)
    if verbose:
        print("Loaded matrix from", os.path.basename(path))
        print("Shape:", matrix.shape)
        print("Example:\n", matrix[:3], end="\n\n")
    return matrix

In [3]:
%cd ../../

/home/shkarupa/Documents/Semester6/Coursework/parallel-gradient-descent


## Generate data

In [4]:
!python demo/gen_data.py --help

Generate train and test data for linear regression model

Tool arguments:
  -h, --help          show this help message and exit
  -t , --n-train      number of train samples (default: 1000)
  -e , --n-eval       number of evaluation samples (default: 100)
  -f , --n-features   number of per sample features (default: 10)
  -s , --seed         seed that make data generation deterministic (default:
                      None)
  -o , --out-dir      path to directory where generated data will be stored
                      (default: ./data)


In [5]:
!python demo/gen_data.py --n-train 500 --n-eval 100 --n-features 12 --seed 42 --out-dir data

## Look at data

In [6]:
!ls data

x_eval.npy  x_train.npy  y_eval.npy  y_train.npy


In [7]:
x_train = load_matrix("data/x_train.npy")
y_train = load_matrix("data/y_train.npy")

x_eval = load_matrix("data/x_eval.npy")
y_eval = load_matrix("data/y_eval.npy")

Loaded matrix from x_train.npy
Shape: (500, 12)
Example:
 [[ 0.73447675  2.14582493 -1.98196449 -1.35375467 -1.4991812  -1.16948063
  -1.03799135 -2.07711985  2.77996362 -0.72097011 -0.23527391 -1.54493592]
 [-0.20358036 -0.64927755  0.03408347 -0.37912774 -0.76997323  1.67770081
   1.62839662 -0.58168091  0.56898308 -0.55358824 -1.01475673 -1.22394027]
 [-0.8946073   1.44697788  1.03184454  1.25575613 -1.48556037 -0.43449623
  -0.47874862 -0.18687164  0.22213377 -0.30917212 -0.43973106  0.19655478]]

Loaded matrix from y_train.npy
Shape: (500, 1)
Example:
 [[-376.28563841]
 [ 119.31908989]
 [ -81.10116051]]

Loaded matrix from x_eval.npy
Shape: (100, 12)
Example:
 [[-0.92406281  0.38574997  0.24087676 -0.71998953 -0.66569346  0.83315777
  -2.70754386 -1.25350601  0.86424838 -0.6302493  -0.21495484  0.70266   ]
 [ 1.28093958 -1.2699986   0.67194461 -1.49725765  0.38420207 -0.23591653
   0.29497146 -0.6654223  -0.53475667 -0.40688524  0.72190349  0.66884013]
 [ 1.32130403 -1.86653995 -0

## Run Linear regression

In [8]:
!./build/bin/gradient_descent -h

Linear regression config:

CLI options:
  -h [ --help ]                        produce help message
  -c [ --config ] arg                  path to config file

Algorithm options:
  -i [ --input-path ] arg              path to input NPY file
  -t [ --target-path ] arg             path to target NPY file
  -e [ --eval-path ] arg               path to evaluation NPY file
  -o [ --out-path ] arg (=output.npy)  path to output NPY file
  -p [ --parallel ]                    wether to use parallel or serial SGD
  -n [ --num-epochs ] arg (=1000)      number of training epochs
  -l [ --lr ] arg (=0.001)             learning rate
  -w [ --weight-decay ] arg (=0.01)    L2 regularization lambda term
  --normalize                          wether to normalize input
  --num-threads arg (=12)              number of threads to use for parallel 
                                       SGD
  --num-step-epochs arg (=1)           number of epochs to compute in each 
                                       th

### SGD

In [9]:
!cat examples/serial_config.cfg

input-path = data/x_train.npy
target-path = data/y_train.npy
eval-path = data/x_eval.npy
out-path = output/serial_pred.npy
parallel = false
num-epochs = 10000
lr = 0.001
weight-decay = 0.01
normalize = true


In [10]:
!./build/bin/gradient_descent --config examples/serial_config.cfg

SGD: 151 ms


In [11]:
y_pred_sgd = load_matrix("output/serial_pred.npy", verbose=False)

In [12]:
print("MSE:", mean_squared_error(y_pred_sgd.squeeze(), y_eval))

MSE: 13.708756140309415


### Parallel SGD

In [13]:
!cat examples/parallel_config.cfg

input-path = data/x_train.npy
target-path = data/y_train.npy
eval-path = data/x_eval.npy
out-path = output/parallel_pred.npy
parallel = true
num-epochs = 10000
lr = 0.001
weight-decay = 0.01
normalize = true
num-threads = 12
num-step-epochs = 4


In [14]:
# !./build/bin/gradient_descent --config examples/parallel_config.cfg

In [15]:
y_pred_parallel_sgd = load_matrix("output/parallel_pred.npy", verbose=False)

In [16]:
print("MSE:", mean_squared_error(y_pred_parallel_sgd, y_eval))

MSE: 41119.08393241694


### Sklearn

In [17]:
# regressor = make_pipeline(StandardScaler(), LinearRegression())
regressor = make_pipeline(
    StandardScaler(), 
    SGDRegressor(alpha=0.01, learning_rate="constant", eta0=0.001),
)

In [18]:
# start = time.perf_counter_ns()
regressor.fit(x_train, y_train.squeeze())
# end = time.perf_counter_ns()
# print("SGD:", (end - start) / 1e6, "ms")

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdregressor',
                 SGDRegressor(alpha=0.01, eta0=0.001,
                              learning_rate='constant'))])

In [19]:
y_pred = regressor.predict(x_eval)

In [20]:
print("MSE:", mean_squared_error(y_pred, y_eval))

MSE: 16.062651132437246
